# Import packages

In [1]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

from torch.utils.tensorboard import SummaryWriter

In [2]:
df_all = pd.read_excel(r'D:\code\PIE\data\data_all.xlsx')

In [3]:
def same_seed(seed):
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 5201314,     
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'test_ratio': 0.5,
    'n_epochs': 100,     # Number of epochs.
    'batch_size': 32,
    'learning_rate': 1e-5,
    'early_stop': 10,    # If model has not improved for this many consecutive epochs, stop training.
    'save_path': 'D:\code\PIE\models\model.ckpt'
}

In [5]:
# Select important features
columns_to_select = ['Maximum Input Offset Voltage', 'Maximum Single Supply Voltage', 'Minimum Single Supply Voltage', 'Number of Channels per Chip', 'Supplier_Package', 'Typical Gain Bandwidth Product', 
'Maximum Input Offset Voltage_comp', 'Maximum Single Supply Voltage_comp', 'Minimum Single Supply Voltage_comp', 'Number of Channels per Chip_comp', 'Supplier_Package_comp', 'Typical Gain Bandwidth Product_comp',
'Closeness']
new_df = df_all[columns_to_select]

In [6]:
# one-hot encode for the column 'Supplier_Package'
new_df = pd.get_dummies(new_df, columns=['Supplier_Package', 'Supplier_Package_comp'], prefix=['Supplier_Package', 'Supplier_Package_comp'])

In [7]:
# check the data
print(new_df.columns)

Index(['Maximum Input Offset Voltage', 'Maximum Single Supply Voltage',
       'Minimum Single Supply Voltage', 'Number of Channels per Chip',
       'Typical Gain Bandwidth Product', 'Maximum Input Offset Voltage_comp',
       'Maximum Single Supply Voltage_comp',
       'Minimum Single Supply Voltage_comp',
       'Number of Channels per Chip_comp',
       'Typical Gain Bandwidth Product_comp', 'Closeness',
       'Supplier_Package_DFN', 'Supplier_Package_DFN EP',
       'Supplier_Package_MSOP', 'Supplier_Package_Mini-SO',
       'Supplier_Package_SC-70', 'Supplier_Package_SO',
       'Supplier_Package_SO EP', 'Supplier_Package_SO N',
       'Supplier_Package_SO W', 'Supplier_Package_SOIC',
       'Supplier_Package_SOP-J', 'Supplier_Package_SOT-23',
       'Supplier_Package_TSSOP', 'Supplier_Package_comp_CDIP',
       'Supplier_Package_comp_DFN EP', 'Supplier_Package_comp_DFN-U1 EP',
       'Supplier_Package_comp_DSBGA', 'Supplier_Package_comp_EMP',
       'Supplier_Package_comp_ESON

In [8]:
# reorganize the columns

new_df = new_df[['Maximum Input Offset Voltage', 'Maximum Single Supply Voltage',
       'Minimum Single Supply Voltage', 'Number of Channels per Chip',
       'Typical Gain Bandwidth Product', 'Supplier_Package_DFN', 'Supplier_Package_DFN EP',
       'Supplier_Package_MSOP', 'Supplier_Package_Mini-SO',
       'Supplier_Package_SC-70', 'Supplier_Package_SO',
       'Supplier_Package_SO EP', 'Supplier_Package_SO N',
       'Supplier_Package_SO W', 'Supplier_Package_SOIC',
       'Supplier_Package_SOP-J', 'Supplier_Package_SOT-23',
       'Supplier_Package_TSSOP', 
       'Maximum Input Offset Voltage_comp',
       'Maximum Single Supply Voltage_comp',
       'Minimum Single Supply Voltage_comp',
       'Number of Channels per Chip_comp',
       'Typical Gain Bandwidth Product_comp',
       'Supplier_Package_comp_CDIP',
       'Supplier_Package_comp_DFN EP', 'Supplier_Package_comp_DFN-U1 EP',
       'Supplier_Package_comp_DSBGA', 'Supplier_Package_comp_EMP',
       'Supplier_Package_comp_ESON', 'Supplier_Package_comp_ESON-U1',
       'Supplier_Package_comp_FLP', 'Supplier_Package_comp_HSOIC EP',
       'Supplier_Package_comp_HTSSOP EP', 'Supplier_Package_comp_HVSSOP EP',
       'Supplier_Package_comp_LFCSP EP', 'Supplier_Package_comp_MFP',
       'Supplier_Package_comp_MSOP', 'Supplier_Package_comp_MSOP EP',
       'Supplier_Package_comp_Micro', 'Supplier_Package_comp_Mini-SO',
       'Supplier_Package_comp_PSOP', 'Supplier_Package_comp_SC-70',
       'Supplier_Package_comp_SC-74A', 'Supplier_Package_comp_SC-88A',
       'Supplier_Package_comp_SNT-A', 'Supplier_Package_comp_SO',
       'Supplier_Package_comp_SO N', 'Supplier_Package_comp_SOIC',
       'Supplier_Package_comp_SOIC N', 'Supplier_Package_comp_SOIC N EP',
       'Supplier_Package_comp_SOIC W', 'Supplier_Package_comp_SOP',
       'Supplier_Package_comp_SOP EP', 'Supplier_Package_comp_SOP-J',
       'Supplier_Package_comp_SOT-23', 'Supplier_Package_comp_SOT-25',
       'Supplier_Package_comp_SOT-353', 'Supplier_Package_comp_SSOP',
       'Supplier_Package_comp_SSOP-B', 'Supplier_Package_comp_TDFN EP',
       'Supplier_Package_comp_TMSOP', 'Supplier_Package_comp_TSOP',
       'Supplier_Package_comp_TSOT', 'Supplier_Package_comp_TSOT-23',
       'Supplier_Package_comp_TSSOP', 'Supplier_Package_comp_TSSOP EP',
       'Supplier_Package_comp_TSSOP W', 'Supplier_Package_comp_TSSOP-B',
       'Supplier_Package_comp_TSSOP-BJ', 'Supplier_Package_comp_TVSP',
       'Supplier_Package_comp_UDFN EP', 'Supplier_Package_comp_VSON EP',
       'Supplier_Package_comp_VSP', 'Supplier_Package_comp_VSSOP',
       'Supplier_Package_comp_WSON EP', 'Supplier_Package_comp_uMAX',
       'Closeness']]

In [9]:
print(new_df.columns)

Index(['Maximum Input Offset Voltage', 'Maximum Single Supply Voltage',
       'Minimum Single Supply Voltage', 'Number of Channels per Chip',
       'Typical Gain Bandwidth Product', 'Supplier_Package_DFN',
       'Supplier_Package_DFN EP', 'Supplier_Package_MSOP',
       'Supplier_Package_Mini-SO', 'Supplier_Package_SC-70',
       'Supplier_Package_SO', 'Supplier_Package_SO EP',
       'Supplier_Package_SO N', 'Supplier_Package_SO W',
       'Supplier_Package_SOIC', 'Supplier_Package_SOP-J',
       'Supplier_Package_SOT-23', 'Supplier_Package_TSSOP',
       'Maximum Input Offset Voltage_comp',
       'Maximum Single Supply Voltage_comp',
       'Minimum Single Supply Voltage_comp',
       'Number of Channels per Chip_comp',
       'Typical Gain Bandwidth Product_comp', 'Supplier_Package_comp_CDIP',
       'Supplier_Package_comp_DFN EP', 'Supplier_Package_comp_DFN-U1 EP',
       'Supplier_Package_comp_DSBGA', 'Supplier_Package_comp_EMP',
       'Supplier_Package_comp_ESON', 'Supplier_

In [10]:
new_df = new_df.values

# Dataloader

In [11]:
class PIEDataset(Dataset):
    '''
    x: Features.
    y: Targets.
    '''
    def __init__(self, x, y=None):
        self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

In [12]:
def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set))
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

In [13]:
train_data, valid_data_all = train_valid_split(new_df, config['valid_ratio'], config['seed'])
valid_data, test_data = train_valid_split(valid_data_all, config['test_ratio'], config['seed'])

# Print out the data size.
print(f"""train_data size: {train_data.shape}
valid_data size: {valid_data.shape}
test_data size: {test_data.shape}""")

y_train, y_valid, y_test = train_data[:,-1], valid_data[:,-1], test_data[:,-1] # label
x_train, x_valid, x_test = train_data[:,:-1], valid_data[:,:-1], test_data[:,:-1] # features

x_train = torch.FloatTensor(x_train.astype(np.float32))
x_valid = torch.FloatTensor(x_valid.astype(np.float32))
x_test = torch.FloatTensor(x_test.astype(np.float32))
y_train = torch.FloatTensor(y_train.astype(np.float32))
y_valid = torch.FloatTensor(y_valid.astype(np.float32))
y_test = torch.FloatTensor(y_test.astype(np.float32))

train_dataset, valid_dataset, test_dataset = PIEDataset(x_train, y_train), PIEDataset(x_valid, y_valid), PIEDataset(x_test, y_test)

# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

train_data size: (48000, 77)
valid_data size: (6000, 77)
test_data size: (6000, 77)


# Deep Learning Model

This deep learning model is an example I built and contains only linear layers, just for testing.

In [42]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        self.layers = nn.Sequential(

            nn.Linear(input_dim, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Linear(32, 16),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Linear(16, 8),
            nn.BatchNorm1d(8),
            nn.ReLU(),

            nn.Linear(8, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1) # (B, 1) -> (B)
        return x

Training Loop

In [ ]:
def trainer(train_loader, valid_loader, model, config, device):

    criterion = nn.MSELoss(reduction='mean') 

    # optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.9)
    # optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'], weight_decay=0.01)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate'], weight_decay=0.005)
    # optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.9, nesterov=True, weight_decay=0.01)

    writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train() # Set the model to train mode.
        loss_record = []

        # tqdm is a package to visualize the training progress.
        train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_pbar:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move data to device.
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            loss_record.append(loss.detach().item())

            # Display current epoch number and loss on tqdm progress bar.
            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval() # Set the model to evaluation mode.
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())

        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save the best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else:
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return

In [43]:
model = My_Model(input_dim=x_train.shape[1]).to(device)
trainer(train_loader, valid_loader, model, config, device)

Epoch [1/100]: 100%|██████████| 1500/1500 [00:07<00:00, 213.60it/s, loss=0.543]


Epoch [1/100]: Train loss: 0.6038, Valid loss: 0.4805
Saving model with loss 0.481...


Epoch [2/100]: 100%|██████████| 1500/1500 [00:07<00:00, 196.66it/s, loss=0.39] 


Epoch [2/100]: Train loss: 0.4599, Valid loss: 0.4176
Saving model with loss 0.418...


Epoch [3/100]: 100%|██████████| 1500/1500 [00:07<00:00, 197.82it/s, loss=0.355]


...

Epoch [26/100]: Train loss: 0.0097, Valid loss: 0.0066


Epoch [27/100]: 100%|██████████| 1500/1500 [00:07<00:00, 210.28it/s, loss=0.00819]


Epoch [27/100]: Train loss: 0.0096, Valid loss: 0.0066


Epoch [28/100]: 100%|██████████| 1500/1500 [00:07<00:00, 208.49it/s, loss=0.00994]


Epoch [28/100]: Train loss: 0.0093, Valid loss: 0.0066

Model is not improving, so we halt the training session.


In [44]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

Reusing TensorBoard on port 6007 (pid 15412), started 1:10:38 ago. (Use '!kill 15412' to kill it.)

Testing

In [45]:
model.eval()
criterion = nn.MSELoss(reduction='mean')
loss_record = []

for x, y in test_loader:
  x, y = x.to(device), y.to(device)
  with torch.no_grad():
    pred = model(x)
    loss = criterion(pred, y)

  loss_record.append(loss.item())

mean_test_loss = sum(loss_record)/len(loss_record)
print(f'Test loss: {mean_test_loss:.4f}')

Test loss: 0.0064


Is this performance good or bad? Considering the unclear evaluation criteria, I think more information is needed.

# Random Forest

In [14]:
# sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Random Forest Regression Model
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# Training
rf_regressor.fit(x_train, y_train)

# Testing
y_pred = rf_regressor.predict(x_test)

# Evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("MSE:", mse)
print("R²:", r2)

MSE: 0.0005366648237340567
R²: 0.9462159001061905


# Gradient Boosting Regressor

In [29]:
# sklearn
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

# Hyperparameters
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
}

# GridSearchCV
grid_search = GridSearchCV(estimator=GradientBoostingRegressor(random_state=42),
                           param_grid=param_grid,
                           cv=5,
                           scoring='neg_mean_squared_error')

# Training
grid_search.fit(x_train, y_train)

# Best model
best_gbr = grid_search.best_estimator_

# Testing
y_pred = best_gbr.predict(x_test)

# Evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Best MSE:", mse)
print("Best R²:", r2)

Best MSE: 0.0012307617629988524
Best R²: 0.8766540852332547


# Support Vector Regression

In [ ]:
# sklearn
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

# Hyperparameters
param_grid = {
    'C': [0.1, 1],  # Regularization parameter (the complexity of the model)
    'epsilon': [0.01, 0.1],
    'kernel': ['linear','poly']  # Kernel function
}

# GridSearchCV
grid_search = GridSearchCV(estimator=SVR(), param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# Training
grid_search.fit(x_train, y_train)

# Best model
best_svr = grid_search.best_estimator_

# Testing
y_pred = best_svr.predict(x_test)

# Evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Best MSE:", mse)
print("Best R²:", r2) 